## List of Citys:

- New York, NY
- Los Angeles, CA
- Chicago, IL
- Seattle, WA
- Dallas, TX

## TBD
- Houston, TX
- Washington, DC
- Philadelphia, PA
- Atlanta, GA
- Miami, FL
- Phoenix, AZ
- Boston, MA
- Riverside, CA
- San Francisco, CA
- Detroit, MI


## Collected Data

- New York Crime Data 2000-2023 (Yearly)
  - Convert to rate per 100,000 population
  - Try find Monthly data
- Los Angeles Crime Data 2010-2024 (Individual Records)
  - Filter and only keep major crime
  - Aggregate to Monthly
  - Try find data before 2010
- Chicago Crime Data 2001 - Present (Inidividual Records)
  - Filter and only keep major crime
  - Aggregate to Monthly
- Dallas Historical Violent Crime Rate per 100,000 Population (Yearly)
  - Try find Motnhly Data
  - Try find 2018-present
- Huston Crime Data 2009-2024(Individual Recrods)
  - Filter and only keep major crime
  - Aggregate to Monthly
  - https://www.houstontx.gov/police/cs/crime-stats-archives.htm
- Washinton DC
- Philadelphia Crime Data 2006-2022, 2024 (Individual Records)
  - Find 2023
  - Aggregate to Monthly
- Atlant Crime Data 2009-2024 (Inidividual Records)
  - Find earlier
  - Aggredate to Monthly
- Miami Crime Data Per 100,000 population 2013 - 2022
  - Find other years
  - Fileter out Miami Metro
- Phoenix Crime Data 2015-2024 (Inidividual Records)
  - Find earlier
  - Aggredate to Monthly
- Boston Crime Data per 100,000 population 1999-2018 (yearly)
  - Find other years
  - Find more detailed records
  - https://data.boston.gov/dataset/crime-incident-reports-august-2015-to-date-source-new-system  
-

## I found there is a crime index from city-data.com; We can directly use this index as our feature

Below is a scraper to collect the Violent crime rate and Propert Crime rate for each city

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def get_crime_data(url):
    # Function to extract crime data from a given div
    def extract_crime_data(crime_div):
        crime_data = {}
        years = []

        hgraph_divs = crime_div.find_all('div', class_='hgraph')

        for hgraph in hgraph_divs:
            year_text = hgraph.find('b').get_text(strip=True)
            year = year_text.split()[-1]
            years.append(year)

            rows = hgraph.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 2:
                    city_name = cells[0].get_text(strip=True).replace(':', '')
                    city_rate = cells[1].get_text(strip=True)

                    if city_name not in crime_data:
                        crime_data[city_name] = {}
                    crime_data[city_name][year] = city_rate

        return crime_data, years

    # Send a GET request to the website
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the divs with ids 'violent-crime' and 'property-crime'
        violent_crime_div = soup.find('div', id='violent-crime')
        property_crime_div = soup.find('div', id='property-crime')

        all_data = []
        all_years = set()

        if violent_crime_div:
            violent_crime_data, violent_crime_years = extract_crime_data(violent_crime_div)
            all_data.append((violent_crime_data, 'Violent Crime'))
            all_years.update(violent_crime_years)

        if property_crime_div:
            property_crime_data, property_crime_years = extract_crime_data(property_crime_div)
            all_data.append((property_crime_data, 'Property Crime'))
            all_years.update(property_crime_years)

        # Create DataFrame
        rows = []
        for data, crime_type in all_data:
            for city, rates in data.items():
                row = {'City': city, 'Type': crime_type}
                for year in all_years:
                    row[year] = rates.get(year, None)
                rows.append(row)

        df = pd.DataFrame(rows, columns=['City', 'Type'] + sorted(all_years))

        return df
    else:
        raise Exception(f"Failed to retrieve the webpage. Status code: {response.status_code}")

In [ ]:
cities_urls = {
    'New York, NY': 'https://www.city-data.com/crime/crime-New-York-New-York.html',
    'Los Angeles, CA': 'https://www.city-data.com/crime/crime-Los-Angeles-California.html',
    'Chicago, IL': 'https://www.city-data.com/crime/crime-Chicago-Illinois.html',
    'Dallas, TX': 'https://www.city-data.com/crime/crime-Dallas-Texas.html',
    'Houston, TX': 'https://www.city-data.com/crime/crime-Houston-Texas.html',
    'Washington, DC': 'https://www.city-data.com/crime/crime-Washington-District-of-Columbia.html',
    'Philadelphia, PA': 'https://www.city-data.com/crime/crime-Philadelphia-Pennsylvania.html',
    'Atlanta, GA': 'https://www.city-data.com/crime/crime-Atlanta-Georgia.html',
    'Miami, FL': 'https://www.city-data.com/crime/crime-Miami-Florida.html',
    'Phoenix, AZ': 'https://www.city-data.com/crime/crime-Phoenix-Arizona.html',
    'Boston, MA': 'https://www.city-data.com/crime/crime-Boston-Massachusetts.html',
    'Riverside, CA': 'https://www.city-data.com/crime/crime-Riverside-California.html',
    'San Francisco, CA': 'https://www.city-data.com/crime/crime-San-Francisco-California.html',
    'Detroit, MI': 'https://www.city-data.com/crime/crime-Detroit-Michigan.html',
    'Seattle, WA': 'https://www.city-data.com/crime/crime-Seattle-Washington.html'
}

In [ ]:
combined_df = pd.DataFrame()

# Iterate over each city and parse data
for city, url in cities_urls.items():
    city_df = get_crime_data(url)
    combined_df = pd.concat([combined_df, city_df], ignore_index=True)

# Remove duplicate rows for U.S. Average
combined_df = combined_df.drop_duplicates(subset=['City', 'Type'], keep='first')
combined_df.reset_index(drop=True, inplace=True)

In [ ]:
combined_df

,City,Type,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2022,2021
0,New York,Violent Crime,312.5,296.0,282.7,304.9,289.3,312.7,327.9,334.2,324.6,317.2,312.2,306.4,288.3,292.2,307.6,310.9,396.0,NaN
1,U.S. Average,Violent Crime,227.0,232.6,229.1,222.5,208.9,195.7,187.1,187.4,214.2,211.1,219.3,227.3,226.2,221.2,219.9,228.2,219.8,NaN
2,New York,Property Crime,145.8,135.6,127.8,140.5,130.8,130.9,130.2,129.7,123.1,117.4,109.3,101.6,98.4,100.7,97.2,114.2,150.5,NaN
3,U.S. Average,Property Crime,322.3,317.3,309.2,302.2,285.6,276.4,273.5,267.3,250.4,230.8,221.5,216.6,206.4,190.2,179.7,169.6,166.5,NaN
4,Los Angeles,Violent Crime,376.1,360.9,328.6,372.0,336.6,307.0,283.6,264.3,232.6,269.7,356.0,399.7,421.8,414.8,404.0,398.9,459.6,NaN
5,Los Angeles,Property Crime,265.6,237.5,227.2,266.5,242.1,232.1,220.6,216.3,208.6,199.6,219.9,229.3,237.1,228.1,209.7,212.0,261.5,NaN
6,Chicago,Violent Crime,534.5,529.6,527.3,631.3,586.9,550.9,519.8,512.3,519.6,491.3,506.2,626.7,618.2,569.6,532.4,561.4,330.1,175.4
7,Chicago,Property Crime,377.5,370.1,362.2,436.7,405.2,439.3,437.2,396.4,324.3,273.5,257.3,280.4,277.5,261.5,236.5,196.0,279.7,102.6
8,Dallas,Violent Crime,554.1,535.8,475.9,486.9,434.2,422.0,374.4,375.7,370.6,376.7,392.8,429.7,437.1,424.5,484.6,499.8,426.6,458.7
9,Dallas,Property Crime,616.2,587.0,593.2,641.0,583.4,583.0,547.9,470.0,440.6,372.6,358.5,351.8,329.0,334.0,347.8,352.5,373.8,337.9


In [ ]:
file_path = '/content/drive/MyDrive/MADS Capstone Team 23/Data/processed/processed_crime_data.csv'
combined_df.to_csv(file_path, index=False)